# README

---

Moodle環境の構築を行います。

## 構成について

### コンテナ構成

![コンテナ構成](images/moodle-000-01.png)

このアプリケーションテンプレートではDockerコンテナを用いて[Moodle](https://moodle.org/)環境の構築を行います。
構築する環境は次の２つのコンテナで構成されます。

* Moodle
  - MoodleのPHPスクリプトを実行するコンテナ
* [MariaDB](https://hub.docker.com/_/mariadb/)
  - Moodleの設定を保存するデータベース

ここでは小規模構成を想定して２つのコンテナを１つのノードに配置する構成とします。

### リバースプロキシ

Moodle環境へのアクセスをリバースプロキシを経由するように構成することが可能です。ただし、リバースプロキシの構築自体についてはこのアプリケーションテンプレートの対象外となります。

![リバースプロキシ構成](images/moodle-000-02.png)

### HTTPS接続

デフォルトの構成ではMoodleコンテナへの接続はHTTPとなっています。Moodleコンテナにmod_sslパッケージやサーバ証明書などを追加しMoodleへの接続方法をHTTPSに変更することができます。

![HTTPS構成](images/moodle-000-03.png)

HTTPS接続でMoodle環境を構築するには、まずHTTP接続構成で構築する「021-Moodleコンテナの起動.ipynb」を実行した後に「051-HTTPS接続に変更する.ipynb」を実行してください。

### LDAP認証

MoodleのLDAP認証プラグインを有効にしてLDAP認証を利用することができます。

![LDAP認証](images/moodle-000-04.png)

LDAPサーバに接続するためにCA証明書の追加やDNSサーバ設定の変更が必要となる場合は「211-LDAP認証の設定.ipynb」を実行してください。

### ログの転送

MoodleコンテナのログにはApache HTTP Serverのアクセスログなどが出力されています。ログを集約するために構築環境外の syslogサーバにログを送信する構成をとることができます。

![syslog構成](images/moodle-000-05.png)

コンテナログの転送にはDockerのロギングドライバの変更を行います。「041-ロギングドライバを変更する.ipynb」を実行してください。

## 準備事項

このアプリケーションテンプレートを実行する前に、必要となる準備事項について記します。

### 仮想マシン作成に必要となるもの

このアプリケーションテンプレートではMoodle環境を構築するための仮想マシンの作成に以下の３つの方法を選択できます。

1. VCP SDKを利用する
1. Amazon EC2インスタンスを利用する
1. Azure仮想マシンを利用する

どの方法を選択するかによって準備するものが異なります。

#### VCP SDKを利用する場合

* VCP SDKのセットアップ
* VC Controllerのアクセストークン

#### Amazon EC2インスタンスを利用する場合

* Amazon のクレデンシャル（アクセスキー、シークレットキー）
* Amazon VPCとサブネットが作成済であること
* EC2キーペアが作成済であること

#### Azure 仮想マシンを利用する場合

* Azureのサービスプリンシパル（サブスクリプションID、 テナントID、クライアントID、シークレット）
* 仮想ネットワーク、サブネットが作成済であること

#### 共通で必要となるもの

* アプリケーションテンプレートで作成した仮想マシンにSSH公開鍵認証でログインするための公開鍵、秘密鍵

### Moodle環境構築に必要となるもの

* Moodleを公開するときのURL
* リバースプロキシに関する情報（リバースプロキシを利用する場合）
* サーバ証明書(HTTPS接続で公開する場合)

## Notebookの一覧

お手本Notebookの一覧を示します。

**注意**:

この節ではお手本Notebookへのリンクを示す箇所がありますが、リンク先のNotebookは参照用となっていて**そのままでは実行できません**。

> Notebook自体は実行できてしまいますが、パスなどが想定しているものと異なるため正しく処理できずエラーとなります。

次のどちらかの手順で作業用Notebookを作成する必要があります。

* 次節の「作業用Notebookの作成」で作業用のNotebookを作成する。
* お手本Notebookを配置してある `notebooks/` から、この`000-README.ipynb`と同じディレクトリにNotebookをコピーする。

### 各Notebookの関連について

各Notebookの実行順序などの関係性を示す図を表示します。

図を表示するためのライブラリ `blockdiag` をインストールします。

In [ ]:
!pip install blockdiag --user

次のセルを実行すると、各Notebookの関連を示す図を表示します。

> 図が表示されずに `<IPython.core.display.SVG object>` と表示されている場合は、次のセルを `unfreeze` した後に再実行してください。

図に表示される１つのブロックが１つのNotebookに対応しており、ブロックのタイトル部分にNotebookへのリンクが埋め込まれています。

In [ ]:
from IPython.display import SVG
%run scripts/nb_utils.py
setup_diag()
SVG(filename=generate_svg_diag(diag='images/notebooks.diag'))

### 各Notebookの目次

次のセルを実行すると、各お手本Notebookの目次が表示されます。

> 目次が表示されずに `<IPython.core.display.Markdown object>` と表示されている場合は、次のセルを `unfreeze` した後に再実行してください。

リンクが表示されている項目が一つのNotebookに対応しており、そのサブ項目が各Notebook内の目次になっています。

In [ ]:
from IPython.display import Markdown
%run scripts/nb_utils.py
Markdown(notebooks_toc())

## 作業用Notebookの作成

この節のセルを実行することで、お手本Notebookから作業用Notebookを作成することができます。

まず、作業用Notebookを配置するディレクトリを指定してください。

In [ ]:
WORK_DIR = 'work'

次のセルを実行すると、Notebook名のドロップダウンリストと「作業開始」ボタンが現れます。
「作業開始」ボタンを押すと、お手本Notebookのコピーを作成した後、自動的にブラウザでコピーが開きます。
Notebookの説明を確認しながら実行、適宜修正しながら実行していってください。

> このNotebookを Shutdown した後に再度開いた場合、次のセルに既に表示されている「作用開始」ボタンが正しく動作しません。この節のセルをいったん unfreeze した後、セルを再実行してから「作業開始」ボタンをクリックして下さい。

In [ ]:
from IPython.core.display import HTML
%run scripts/nb_utils.py
setup_nb_workdir(WORK_DIR)
HTML(generate_html_work_nbs(WORK_DIR))